In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
cd /content/drive/My Drive/Computer_vision/Project/banana_classifier

/content/drive/My Drive/Computer_vision/Project/banana_classifier


In [ ]:
ls

banana_classifier_inference.ipynb  data/  models/  requirements.txt


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from torch.autograd import Variable
import cv2
from glob import glob

test_transforms = transforms.Compose([transforms.Resize(224),
                                      transforms.ToTensor(),
                                     ])
class BananaClassifier:
  def __init__(self):
    self.model_banana = torch.load('models/banana_classifier.pth')
    self.model_banana.eval()
    self.model_ripeness = torch.load("models/banana_ripeness.pth")
    self.model_ripeness.eval()
    self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    self.classes_ = ["Banana", "Not_Banana"]
    self.classes_ripeness = ['overripe', 'ripe', 'unripe']
    print("-----Model Loaded-----")
  
  def predict_image(self, image, model):
    image_tensor = test_transforms(image).float()
    image_tensor = image_tensor.unsqueeze_(0)
    input = Variable(image_tensor)
    input = input.to(self.device)
    output = model(input)
    index = output.data.cpu().numpy().argmax()
    return index

  def classify_Banana(self, image):
    index = self.predict_image(image, self.model_banana)
    class_ = self.classes_[index]
    print(class_+":",index)
    return class_

  def classify_ripeness(self, image):
    index = self.predict_image(image, self.model_ripeness)
    class_ = self.classes_ripeness[index]
    print("ripeness:", index, class_)
    return class_ 

  def write_text(self, frame, str, x, y, fontScale, color, thickness):
          cv2.putText(frame, str, (x, y),
                              cv2.FONT_HERSHEY_SIMPLEX, fontScale, color, thickness)     

In [ ]:
%matplotlib inline
images = glob("data/test/overripe/*.jpg")
to_pil = transforms.ToPILImage()
fig=plt.figure(figsize=(100,100))
banana_classifier = BananaClassifier()
for i, path in enumerate(images):
  image = cv2.imread(path)
  image_rgb = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
  image_infer = to_pil(image_rgb)
  Banana = banana_classifier.classify_Banana(image_infer)
  if Banana == "Banana":
     ripeness = banana_classifier.classify_ripeness(image_infer)
  else :
     ripeness = ""

  text =  Banana + ": " + ripeness
  print()
  banana_classifier.write_text(image_rgb, text, 10, 30, 0.8, (255,0,0), 2)
  #cv2.imshow("classifier", image)
  #cv2.waitKey(0)  
  sub = fig.add_subplot(1, len(images), i+1)
  plt.axis('off')
  plt.imshow(image_rgb)
plt.show()


Output hidden; open in https://colab.research.google.com to view.